In [1]:
import os
import pandas as pd
import numpy as np
import collections
import tensorflow as tf
import pickle

In [2]:
number_of_crossvalidation_run = 1
models_folder_name = os.path.join(os.getcwd(),'models','train', str(number_of_crossvalidation_run))
summaries_folder_name = os.path.join(os.getcwd(),'summaries','train',str(number_of_crossvalidation_run))

path_to_preprocessed_texts = os.path.join(models_folder_name, 'recipes_train_dataset.pkl')



df_preprocessed_texts = pd.read_pickle(path_to_preprocessed_texts)
preprocessed_texts = df_preprocessed_texts.preprocessed_texts.values.tolist()
labels = df_preprocessed_texts['labels'].values.tolist()

unique_labels=sorted(set(labels))
number_categories=len(unique_labels)
categories_indices=np.linspace(0,number_categories-1,number_categories,dtype=int)
labels2integers=dict(zip(unique_labels,categories_indices))

print(labels2integers)
print("number of recipes in train dataset: ", df_preprocessed_texts.labels.count())

{'cheesecake': 1, 'hummus': 13, 'chocolate_cake': 4, 'chocolate_mousse': 5, 'deviled_eggs': 6, 'fried_rice': 9, 'sushi': 23, 'nachos': 17, 'macarons': 16, 'pancakes': 19, 'dumplings': 7, 'tacos': 24, 'onion_rings': 18, 'frozen_yogurt': 10, 'garlic_bread': 11, 'chicken_wings': 3, 'macaroni_and_cheese': 15, 'lasagna': 14, 'pizza': 20, 'sashimi': 21, 'apple_pie': 0, 'hamburger': 12, 'french_toast': 8, 'chicken_curry': 2, 'steak': 22, 'waffles': 26, 'tiramisu': 25}
number of recipes in train dataset:  264


In [3]:
batch_size = 8
generations = 200000
model_learning_rate = 0.0005

embedding_size = 27   #word embedding size
doc_embedding_size = 27  #document embedding size
concatenated_size = embedding_size + doc_embedding_size

save_embeddings_every = 5000
print_valid_every = 5000
print_loss_every = 50

In [4]:
def build_dictionary(preprocessed_texts):
    words=[w for words_in_recipe in preprocessed_texts for w in words_in_recipe]
    count = []
    count.extend(collections.Counter(words))
    count=sorted(count)
    word_dict = {}
    for word in count:
        word_dict[word] = len(word_dict)
    
    return (word_dict)

#replace each word in texts with integer value
def text_to_numbers(preprocessed_texts, word_dict):
    data = []
    for prepr_text in preprocessed_texts:
        text_data = []
        for word in prepr_text:
            if word in word_dict:
                word_ix = word_dict[word]
            else:
                word_ix = 0
            text_data.append(word_ix)
        data.append(text_data)
    return (data)


def create_batch_data(text_with_words_conv_to_numbers, batch_size=batch_size):
    batch_data = []
    label_data = []
    
    rand_text_ix = int(np.random.choice(len(text_with_words_conv_to_numbers), size=1))
    rand_text = text_with_words_conv_to_numbers[rand_text_ix]
    word_to_predict_label=np.random.choice(list(set(rand_text)), size=1,replace=False)
    
    while len(batch_data) < batch_size:
        item_in_batch=[]        
        
        label_words=np.random.choice(rand_text, size=1,replace=False)

        item_in_batch.extend(word_to_predict_label)
        item_in_batch.append(rand_text_ix)     
        label_data.extend(label_words)
        batch_data.append(item_in_batch)

        
    batch_data = np.array(batch_data)
    label_data = np.transpose(np.array(label_data))

    return (batch_data, label_data)

In [5]:
word_dictionary=build_dictionary(preprocessed_texts)
vocabulary_size=len(word_dictionary)
print(word_dictionary)
print(vocabulary_size)

word_dictionary_rev = dict(zip(word_dictionary.values(), word_dictionary.keys()))

{'run': 340, 'oven': 271, 'enjoy': 145, 'tongs': 418, 'filet': 152, 'coat': 93, 'egg': 142, 'cumin': 121, 'instruction': 201, 'pressure': 314, 'food': 163, 'oat': 266, 'cardamom': 70, 'heart': 193, 'sandwich': 346, 'skillet': 367, 'chill': 82, 'moisture': 252, 'pot': 307, 'mustard': 257, 'ground': 186, 'block': 41, 'stuffed': 400, 'taco': 405, 'tahini': 406, 'strip': 399, 'boil': 44, 'fryer': 169, 'crispy': 117, 'layer': 211, 'season': 353, 'coffee': 98, 'dry': 139, 'mash': 235, 'chopstick': 86, 'jam': 205, 'ball': 18, 'bundt': 61, 'lemon': 214, 'bit': 36, 'scallion': 351, 'aquafaba': 7, 'drop': 138, 'oregano': 270, 'rice': 333, 'mango': 228, 'mat': 236, 'ladyfinger': 209, 'cooky': 106, 'pico': 292, 'coriander': 109, 'naan': 258, 'buffalo': 59, 'recipe': 330, 'seed': 356, 'cocoa': 95, 'spray': 382, 'wire': 449, 'milliliter': 247, 'foil': 162, 'sugar': 401, 'flip': 159, 'sprig': 384, 'gelatin': 175, 'clove': 92, 'seaweed': 355, 'masala': 233, 'bell': 33, 'basil': 23, 'smith': 371, 'mush

In [6]:
text_data = text_to_numbers(preprocessed_texts, word_dictionary)
print(text_data)

[[6, 293, 131, 6, 310, 63, 401, 89, 345, 282, 142, 275, 6, 63, 401, 89, 265, 345, 358, 319, 282, 142, 142, 275, 289, 6, 441, 310, 310, 441, 251, 63, 401, 89, 265, 345, 445, 251, 6, 394, 282, 312, 282, 331, 6, 251, 331, 441, 331, 6, 331, 331, 154, 445, 142, 293, 358, 293, 15, 55, 331, 275, 331, 6, 293, 145], [6, 293, 131, 319, 282, 63, 401, 6, 89, 186, 358, 282, 63, 401, 6, 249, 319, 282, 370, 6, 63, 282, 401, 282, 249, 6, 282, 145], [69, 6, 293, 330, 131, 181, 371, 6, 207, 161, 401, 186, 89, 186, 293, 119, 142, 116, 55, 401, 63, 181, 371, 6, 207, 401, 186, 89, 186, 293, 119, 142, 116, 63, 345, 312, 271, 6, 370, 207, 161, 401, 89, 6, 293, 119, 293, 6, 370, 47, 119, 293, 119, 6, 370, 135, 119, 293, 142, 293, 119, 55, 282, 349, 116, 401, 401, 445, 63, 44, 69, 69, 348, 107, 293, 345, 370, 145], [104, 115, 6, 293, 330, 131, 115, 348, 6, 293, 154, 67, 249, 63, 432, 198, 116, 115, 6, 293, 67, 249, 63, 432, 198, 116, 115, 348, 6, 293, 104, 47, 67, 249, 104, 6, 115, 370, 63, 370, 249, 432, 198,

In [7]:
valid_words = ['tuna', 'rice', 'sushi', 'roll', 'sashimi','steak','grill', 'sauce', 'cream',
               'cheesecake', 'pizza', 'lasagna', 'hamburger']

valid_examples = [word_dictionary[x] for x in valid_words if x in word_dictionary.keys()]
print(valid_examples)

[428, 333, 403, 337, 347, 391, 185, 348, 116, 77, 300, 210, 190]


In [8]:
batch_data, label_data = create_batch_data(preprocessed_texts)
print(batch_data)
print(label_data)
print(np.shape(label_data))

[['onion' '154']
 ['onion' '154']
 ['onion' '154']
 ['onion' '154']
 ['onion' '154']
 ['onion' '154']
 ['onion' '154']
 ['onion' '154']]
['mac' 'onion' 'yeast' 'powder' 'onion' 'recipe' 'onion' 'onion']
(8,)


In [9]:
sess = tf.InteractiveSession()

In [10]:
print('Creating Model')

embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0), name="word_embeddings")
doc_embeddings = tf.Variable(tf.random_uniform([len(preprocessed_texts), doc_embedding_size], -1.0, 1.0), name="doc_embeddings")
decoder_weights = tf.Variable(tf.truncated_normal([vocabulary_size, concatenated_size],
                                               stddev=1.0 / np.sqrt(concatenated_size)),
                                               name="decoder_weights")
decoder_biases = tf.Variable(tf.zeros([vocabulary_size]), name="decoder_biases")


x_inputs = tf.placeholder(tf.int32, shape=[None, 2]) #2:  1 for word index and 1 for doc index
y_target = tf.placeholder(tf.int32, shape=[batch_size])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

embed= tf.nn.embedding_lookup(embeddings, x_inputs[:, 0])
    
doc_indices = tf.slice(x_inputs, [0,1],[batch_size,1])
doc_embed = tf.nn.embedding_lookup(doc_embeddings,doc_indices)
final_embed = tf.concat([embed, tf.squeeze(doc_embed)],1)

logits = tf.matmul(final_embed, tf.transpose(decoder_weights)) + decoder_biases

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=y_target))
optimizer = tf.train.AdamOptimizer(learning_rate=model_learning_rate)
train_step = optimizer.minimize(loss)

#cosine similarity between words
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True, name="cosine_similarity")

with tf.name_scope("performance"):
    loss_ph = tf.placeholder(tf.float32,shape=None,name='loss_summary')
    loss_summary = tf.summary.scalar('loss', loss_ph)
    performance_summaries = tf.summary.merge([loss_summary])

saver = tf.train.Saver({"embeddings": embeddings, "doc_embeddings": doc_embeddings, "decoder_weights":decoder_weights, "decoder_biases":decoder_biases})
summ_writer = tf.summary.FileWriter(summaries_folder_name, sess.graph)

init = tf.initialize_all_variables()
sess.run(init)


print('Starting Training')

for i in range(generations):
    batch_inputs, batch_labels = create_batch_data(text_data)
    feed_dict = {x_inputs : batch_inputs, y_target : batch_labels}

    sess.run(train_step, feed_dict=feed_dict)

    if (i+1) % print_loss_every == 0:
        loss_val = sess.run(loss, feed_dict=feed_dict)
        summ = sess.run(performance_summaries, feed_dict={loss_ph:loss_val})
        summ_writer.add_summary(summ, i+1)
        print('Loss at step {} : {}'.format(i+1, loss_val))
        
    if (i+1) % print_valid_every == 0:
        sim = sess.run(similarity, feed_dict=feed_dict)
        for j in range(len(valid_words)):
            valid_word = word_dictionary_rev[valid_examples[j]]
            top_k = 5 # number of nearest neighbors
            nearest = (-sim[j, :]).argsort()[1:top_k+1]
            log_str = "Nearest to {}:".format(valid_word)
            for k in range(top_k):
                close_word = word_dictionary_rev[nearest[k]]
                log_str = '{} {},'.format(log_str, close_word)
            print(log_str)
            
    if (i+1) % save_embeddings_every == 0:
        #save vocabulary
        with open(os.path.join(models_folder_name,'doc2vec_recipes_dict_words_integers.pkl'), 'wb') as f:
            pickle.dump(word_dictionary, f)
        
        #save embeddings
        model_checkpoint_path = os.path.join(os.getcwd(),models_folder_name,'doc2vec_recipes_checkpoint.ckpt')
        save_path = saver.save(sess, model_checkpoint_path)
        print('Model saved in file: {}'.format(save_path))
        
sess.close()

Creating Model
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Starting Training
Loss at step 50 : 6.338644981384277
Loss at step 100 : 6.314270496368408
Loss at step 150 : 6.182187557220459
Loss at step 200 : 5.921782493591309
Loss at step 250 : 5.674522399902344
Loss at step 300 : 6.22251033782959
Loss at step 350 : 6.028868675231934
Loss at step 400 : 5.887228012084961
Loss at step 450 : 5.897741794586182
Loss at step 500 : 6.332916259765625
Loss at step 550 : 5.957516670227051
Loss at step 600 : 5.660001277923584
Loss at step 650 : 6.5707621574401855
Loss at step 700 : 5.612837791442871
Loss at step 750 : 6.297659873962402
Loss at step 800 : 5.873055934906006
Loss at step 850 : 5.944061756134033
Loss at step 900 : 5.964816093444824
Loss at step 950 : 5.920680999755859
Loss at step 1000 : 5.87056827545166
Loss at step 1050 : 5.650750637054443
Loss at step 1100 : 5.806759834289551
Loss 

Loss at step 9000 : 5.105460166931152
Loss at step 9050 : 4.978597640991211
Loss at step 9100 : 4.760387420654297
Loss at step 9150 : 4.079474449157715
Loss at step 9200 : 4.113181114196777
Loss at step 9250 : 4.404670715332031
Loss at step 9300 : 4.885677814483643
Loss at step 9350 : 3.4827756881713867
Loss at step 9400 : 4.3529510498046875
Loss at step 9450 : 5.561936378479004
Loss at step 9500 : 5.189963340759277
Loss at step 9550 : 5.380207061767578
Loss at step 9600 : 4.223998069763184
Loss at step 9650 : 3.7235617637634277
Loss at step 9700 : 3.2066707611083984
Loss at step 9750 : 4.291766166687012
Loss at step 9800 : 4.555448055267334
Loss at step 9850 : 3.640852928161621
Loss at step 9900 : 4.270608901977539
Loss at step 9950 : 4.952080249786377
Loss at step 10000 : 3.5648655891418457
Nearest to tuna: virgin, metal, parmesan, beef, nibble,
Nearest to rice: mandu, thigh, crab, spinach, glaze,
Nearest to sushi: circle, cornstarch, cake, mandu, sashimi,
Nearest to roll: worcesters

Loss at step 17400 : 4.355195045471191
Loss at step 17450 : 5.033234596252441
Loss at step 17500 : 3.6915688514709473
Loss at step 17550 : 4.189692497253418
Loss at step 17600 : 4.323850631713867
Loss at step 17650 : 4.113953590393066
Loss at step 17700 : 4.073544502258301
Loss at step 17750 : 3.483029365539551
Loss at step 17800 : 4.71878719329834
Loss at step 17850 : 6.219027519226074
Loss at step 17900 : 4.1261982917785645
Loss at step 17950 : 4.7902398109436035
Loss at step 18000 : 4.206792831420898
Loss at step 18050 : 3.5736303329467773
Loss at step 18100 : 4.531538963317871
Loss at step 18150 : 2.4960105419158936
Loss at step 18200 : 2.9162707328796387
Loss at step 18250 : 4.102075576782227
Loss at step 18300 : 3.6976475715637207
Loss at step 18350 : 4.8748555183410645
Loss at step 18400 : 3.4936137199401855
Loss at step 18450 : 5.206659317016602
Loss at step 18500 : 4.021507263183594
Loss at step 18550 : 4.491116046905518
Loss at step 18600 : 2.0128424167633057
Loss at step 186

Loss at step 25700 : 3.5299153327941895
Loss at step 25750 : 4.786250114440918
Loss at step 25800 : 3.4988746643066406
Loss at step 25850 : 3.86175537109375
Loss at step 25900 : 3.9531044960021973
Loss at step 25950 : 2.161073684692383
Loss at step 26000 : 4.1393866539001465
Loss at step 26050 : 4.860223770141602
Loss at step 26100 : 4.852667331695557
Loss at step 26150 : 3.9258241653442383
Loss at step 26200 : 2.714966058731079
Loss at step 26250 : 3.2622861862182617
Loss at step 26300 : 3.080065965652466
Loss at step 26350 : 3.38969087600708
Loss at step 26400 : 2.7723047733306885
Loss at step 26450 : 2.904503107070923
Loss at step 26500 : 4.451022148132324
Loss at step 26550 : 3.977409839630127
Loss at step 26600 : 4.423873424530029
Loss at step 26650 : 2.9796459674835205
Loss at step 26700 : 2.858307361602783
Loss at step 26750 : 4.012808322906494
Loss at step 26800 : 4.996893882751465
Loss at step 26850 : 3.296677589416504
Loss at step 26900 : 3.6795034408569336
Loss at step 26950

Loss at step 35050 : 3.0607621669769287
Loss at step 35100 : 2.9005136489868164
Loss at step 35150 : 4.201502323150635
Loss at step 35200 : 3.349221706390381
Loss at step 35250 : 3.9242799282073975
Loss at step 35300 : 2.6907718181610107
Loss at step 35350 : 3.0557656288146973
Loss at step 35400 : 2.5199365615844727
Loss at step 35450 : 2.0943939685821533
Loss at step 35500 : 3.6857380867004395
Loss at step 35550 : 3.273022174835205
Loss at step 35600 : 3.565244197845459
Loss at step 35650 : 3.3825998306274414
Loss at step 35700 : 2.950794219970703
Loss at step 35750 : 3.8420560359954834
Loss at step 35800 : 3.7575361728668213
Loss at step 35850 : 3.9630002975463867
Loss at step 35900 : 3.7063212394714355
Loss at step 35950 : 4.701062202453613
Loss at step 36000 : 3.0407538414001465
Loss at step 36050 : 2.6676628589630127
Loss at step 36100 : 3.171696662902832
Loss at step 36150 : 3.3568716049194336
Loss at step 36200 : 2.6609020233154297
Loss at step 36250 : 3.9080519676208496
Loss at

...

Loss at step 164800 : 2.964665412902832
Loss at step 164850 : 2.1993582248687744
Loss at step 164900 : 3.6785531044006348
Loss at step 164950 : 4.168036460876465
Loss at step 165000 : 2.523951530456543
Nearest to tuna: parmesan, metal, eye, position, sushi,
Nearest to rice: thigh, lime, mandu, spinach, simmer,
Nearest to sushi: piece, sesame, sashimi, cut, rice,
Nearest to roll: meal, sheet, oil, crumb, ricotta,
Nearest to sashimi: ring, fryer, seaweed, gyoza, sushi,
Nearest to steak: maple, meat, eye, grate, paste,
Nearest to grill: cider, soy, teriyaki, spring, blender,
Nearest to sauce: jack, wasabi, appetizer, soy, oregano,
Nearest to cream: cherry, chocolate, extract, cake, skinless,
Nearest to cheesecake: cashew, nutmeg, flake, paper, tin,
Nearest to pizza: mixture, white, baguette, flake, run,
Nearest to lasagna: fry, noodle, rare, lemon, teriyaki,
Nearest to hamburger: meat, herb, pine, ½, parsley,
Model saved in file: /notebooks/school/text_feature_extraction/models/train/10/d

Loss at step 172900 : 3.27593731880188
Loss at step 172950 : 2.137934446334839
Loss at step 173000 : 2.7597925662994385
Loss at step 173050 : 3.509080648422241
Loss at step 173100 : 1.6958684921264648
Loss at step 173150 : 3.026787757873535
Loss at step 173200 : 2.1360342502593994
Loss at step 173250 : 2.6058788299560547
Loss at step 173300 : 3.0501904487609863
Loss at step 173350 : 1.916808009147644
Loss at step 173400 : 2.8586344718933105
Loss at step 173450 : 2.7632651329040527
Loss at step 173500 : 3.281456232070923
Loss at step 173550 : 2.6437649726867676
Loss at step 173600 : 3.0593338012695312
Loss at step 173650 : 3.211446762084961
Loss at step 173700 : 3.139158248901367
Loss at step 173750 : 2.772779941558838
Loss at step 173800 : 3.262510299682617
Loss at step 173850 : 2.234438419342041
Loss at step 173900 : 3.856823205947876
Loss at step 173950 : 3.57065486907959
Loss at step 174000 : 2.5673813819885254
Loss at step 174050 : 3.2197928428649902
Loss at step 174100 : 2.8488349

Loss at step 181000 : 2.267538547515869
Loss at step 181050 : 2.5891575813293457
Loss at step 181100 : 4.177131652832031
Loss at step 181150 : 3.3601558208465576
Loss at step 181200 : 2.3578286170959473
Loss at step 181250 : 3.3736257553100586
Loss at step 181300 : 2.7141518592834473
Loss at step 181350 : 2.4655075073242188
Loss at step 181400 : 3.7307815551757812
Loss at step 181450 : 3.2374203205108643
Loss at step 181500 : 2.1318483352661133
Loss at step 181550 : 4.727985858917236
Loss at step 181600 : 3.8297200202941895
Loss at step 181650 : 2.935009002685547
Loss at step 181700 : 2.915163040161133
Loss at step 181750 : 2.756990671157837
Loss at step 181800 : 2.3727216720581055
Loss at step 181850 : 2.5859179496765137
Loss at step 181900 : 2.790215015411377
Loss at step 181950 : 2.4818148612976074
Loss at step 182000 : 2.9674699306488037
Loss at step 182050 : 2.927351951599121
Loss at step 182100 : 3.054628849029541
Loss at step 182150 : 2.975280523300171
Loss at step 182200 : 2.71

Loss at step 190050 : 3.0777182579040527
Loss at step 190100 : 2.855478525161743
Loss at step 190150 : 2.8233797550201416
Loss at step 190200 : 2.8940751552581787
Loss at step 190250 : 3.4715194702148438
Loss at step 190300 : 2.732646942138672
Loss at step 190350 : 2.8079168796539307
Loss at step 190400 : 3.376744508743286
Loss at step 190450 : 3.5214130878448486
Loss at step 190500 : 3.39988374710083
Loss at step 190550 : 2.928256034851074
Loss at step 190600 : 1.8062334060668945
Loss at step 190650 : 2.5178356170654297
Loss at step 190700 : 3.526543617248535
Loss at step 190750 : 2.54960298538208
Loss at step 190800 : 3.069645643234253
Loss at step 190850 : 3.3365318775177
Loss at step 190900 : 3.4135093688964844
Loss at step 190950 : 2.588082790374756
Loss at step 191000 : 3.0972256660461426
Loss at step 191050 : 2.169795036315918
Loss at step 191100 : 2.2765610218048096
Loss at step 191150 : 3.243605375289917
Loss at step 191200 : 2.745587110519409
Loss at step 191250 : 2.741320133

Loss at step 199150 : 3.0190587043762207
Loss at step 199200 : 3.462486982345581
Loss at step 199250 : 3.077979326248169
Loss at step 199300 : 3.108485698699951
Loss at step 199350 : 2.6826486587524414
Loss at step 199400 : 4.088657855987549
Loss at step 199450 : 2.8000659942626953
Loss at step 199500 : 3.6551871299743652
Loss at step 199550 : 3.023803234100342
Loss at step 199600 : 2.169670581817627
Loss at step 199650 : 2.759066104888916
Loss at step 199700 : 2.30612850189209
Loss at step 199750 : 2.341580629348755
Loss at step 199800 : 3.1883344650268555
Loss at step 199850 : 2.194319009780884
Loss at step 199900 : 3.228804111480713
Loss at step 199950 : 2.555026054382324
Loss at step 200000 : 3.11266827583313
Nearest to tuna: parmesan, metal, eye, position, mat,
Nearest to rice: thigh, lime, mandu, simmer, spinach,
Nearest to sushi: piece, sashimi, sesame, rare, bundt,
Nearest to roll: meal, sheet, ricotta, salt, oil,
Nearest to sashimi: fryer, seaweed, sushi, ring, gyoza,
Nearest 